In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnext50_32x4d, resnext101_32x8d
from torch.utils.data import DataLoader
import sys

# Hyperparameters
batch_size = 128
learning_rate = 0.001
num_epochs = 100
confidence_threshold = 0.99  # Confidence threshold (99%)

# Define data transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

# Load CIFAR-100 dataset
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

# Define the ResNeXt model
class CustomResNeXt(nn.Module):
    def __init__(self):
        super(CustomResNeXt, self).__init__()
        self.model = resnext101_32x8d(pretrained=False)  # ResNeXt50 (32x4d) architecture
        self.model.fc = nn.Linear(2048, 100)  # Modify the last fully connected layer for 100 classes (CIFAR-100)

    def forward(self, x):
        return self.model(x)

model = CustomResNeXt().cuda()
import time
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    high_confidence_count = 0  # Count of predictions with confidence > 99%

    for inputs, labels in trainloader:
        inputs, labels = inputs.cuda(), labels.cuda()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # Calculate softmax probabilities
        probabilities = torch.softmax(outputs, dim=1)
        max_probabilities, _ = torch.max(probabilities, dim=1)

        # Count how many have max probability greater than 99%
        high_confidence_count += (max_probabilities > confidence_threshold).sum().item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}, Accuracy: {100.*correct/total}%')
    if (epoch + 1) % 50 == 0:
        elapsed_time = time.time() - start_time
        torch.save(model.state_dict(), '//content//drive//MyDrive//resnet_model_'+str(epoch)+'.pth')
        print(f'Time used after {epoch + 1} epochs: {elapsed_time // 60} minutes and {elapsed_time % 60:.2f} seconds')

    print(f'Number of high-confidence (>99%) predictions: {high_confidence_count} out of {total}')

# Save the trained model
torch.save(model.state_dict(), 'resnet_model.pth')


100%|██████████| 169001437/169001437 [00:02<00:00, 59717225.18it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1, Loss: 4.613860466291228, Accuracy: 4.944%
Number of high-confidence (>99%) predictions: 109 out of 50000
Epoch 2, Loss: 4.176743344577682, Accuracy: 8.96%
Number of high-confidence (>99%) predictions: 85 out of 50000
Epoch 3, Loss: 3.958879217772228, Accuracy: 11.528%
Number of high-confidence (>99%) predictions: 86 out of 50000
Epoch 4, Loss: 3.775374183264535, Accuracy: 14.454%
Number of high-confidence (>99%) predictions: 43 out of 50000
Epoch 5, Loss: 3.722554616001256, Accuracy: 14.986%
Number of high-confidence (>99%) predictions: 34 out of 50000
Epoch 6, Loss: 3.5410894146355827, Accuracy: 17.452%
Number of high-confidence (>99%) predictions: 44 out of 50000
Epoch 7, Loss: 3.322629527362716, Accuracy: 20.188%
Number of high-confidence (>99%) predictions: 33 out of 50000
Epoch 8, Loss: 3.191270174577718, Accuracy: 22.482%
Number of high-confidence (>99%) predictions: 43 out of 50000
Epoch 9, Loss: 3.0478164192355806, Accuracy: 24.916%
Number of high-confidence (>99%) pre

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), '//content//drive//MyDrive//resnet_model_'+str(epoch)+'.pth')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
